In [1]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import locale
#locale.setlocale(locale.LC_ALL, "ru")
locale.setlocale(locale.LC_ALL, "ru_RU.UTF-8")

# import chart_studio
# import chart_studio.plotly as py

# chart_studio.tools.set_credentials_file(username='vityel', api_key='SJLcp4WV9hsofzgws45L')


'ru_RU.UTF-8'

In [2]:
yandex_world = pd.read_csv('yandex_world.csv')
yandex_world.rename(columns = {'Дата':'Date','Страна':'Country','Заражений':'Confirmed','Выздоровлений':'Recovered',
                               'Смертей':'Deaths','Смертей за день':'Day_deaths',
                               'Заражений за день':'Day_confirmed','Выздоровлений за день':'Day_recovered'},inplace = True)
for i in range(yandex_world.shape[0]):
    a= yandex_world.Date[i].split('.')
    yandex_world.Date[i]=datetime.date(int(a[2]),int(a[1]),int(a[0]))

yandex_world.tail(20)

,Country,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths
37567,Япония,2020-07-21,26463,20001,988,757,398,0
37568,Япония,2020-07-22,27136,20482,990,673,481,2
37569,Япония,2020-07-23,28114,20878,992,978,396,2
37570,Япония,2020-07-24,28883,21173,994,769,295,2
37571,Япония,2020-07-25,29684,21464,996,801,291,2
37572,Япония,2020-07-26,30548,21606,998,864,142,2
37573,Япония,2020-07-27,31142,21970,998,594,364,0
37574,Япония,2020-07-28,32116,22636,1001,974,666,3
37575,Япония,2020-07-29,33382,23351,1001,1266,715,0
37576,Япония,2020-07-30,35144,24024,1007,1762,673,6


In [3]:
df3_temp = yandex_world.groupby('Date').sum()
df3_temp['Country']='Весь мир'

y5= pd.concat([yandex_world,df3_temp.reset_index()],axis = 0)
y5.reset_index()
yandex_world = y5
yandex_world.tail(10)

,Country,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths
191,Весь мир,2020-07-31,17599905,10369140,680234,290100,198490,6293
192,Весь мир,2020-08-01,17850617,10553585,685773,250712,184445,5539
193,Весь мир,2020-08-02,18079723,10690555,690065,229106,136970,4292
194,Весь мир,2020-08-03,18282208,10913000,694396,202485,222445,4331
195,Весь мир,2020-08-04,18540789,11134735,701347,258581,221735,6951
196,Весь мир,2020-08-05,18811953,11356275,708424,271164,221540,7077
197,Весь мир,2020-08-06,19097149,11545401,714940,285196,189126,6516
198,Весь мир,2020-08-07,19378036,11737927,721324,280887,192526,6384
199,Весь мир,2020-08-08,19637506,11939109,726781,259470,201182,5457
200,Весь мир,2020-08-09,19861683,12115825,731326,224177,176716,4545


In [4]:
# dt = datetime.datetime.today()
# today = datetime.date(dt.year,dt.month,dt.day)
# ystday = datetime.date(today.year,today.month,today.day-1)
# yst2day = datetime.date(today.year,today.month,today.day-2)
# yst3day = datetime.date(today.year,today.month,today.day-3)

# print(today)
# print(ystday)
# print(yst2day)
# print(yst3day)
myday = list(yandex_world.tail(1).Date)[0]

print(yandex_world.iloc[-1,0])
print('Выбран день: ',myday)

Весь мир
Выбран день:  2020-08-09


In [5]:
X = pd.DataFrame()
country_dict = {}
yandex_world['Rt'] = 1
yandex_world['Rt2'] = 1
yandex_world['Rate_conf'] = 0
yandex_world['Rate_ill'] = 0

temp_df = yandex_world.groupby(['Country','Date']).sum()

# Ниже enumerate  для составления словаря из имен регионов {i:j} , для расчетов можно было использовать только j

for i,j in enumerate(temp_df.index.get_level_values(0).unique()):
    temp_df2 =  temp_df.loc[j]
    temp1 = temp_df2['Day_confirmed'].rolling(4).sum()/(temp_df2["Day_confirmed"].rolling(8).sum()-

                                                              temp_df2["Day_confirmed"].rolling(4).sum())
    temp2 = temp_df2['Confirmed'].rolling(4).sum()/(temp_df2['Confirmed'].rolling(8).sum()-

                                                              temp_df2['Confirmed'].rolling(4).sum())
    temp_df2['Rt'] = round(temp1,3)
    temp_df2['Rt2'] = round(temp2,3)
    temp_df2['Country'] = j
    temp_df2['Remaining_ill']=temp_df2['Confirmed']-temp_df2['Recovered']-temp_df2['Deaths']
    temp_df2['MA7_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=7).mean()
    temp_df2['MA14_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=14).mean()
    temp_df2['MA7_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=7).mean()
    temp_df2['MA14_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=14).mean()
    
    temp_df2['Change_dayconf'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_dayconf'].iloc[k]=temp_df2['Day_confirmed'].iloc[k]-temp_df2['Day_confirmed'].iloc[k-1]
    temp_df2['Change_remill'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_remill'].iloc[k]=temp_df2['Remaining_ill'].iloc[k]-temp_df2['Remaining_ill'].iloc[k-1]
    
    

   #Генератор DF  на каждый регион(сидит в цикле):
   # globals()['x' + str(i)] = temp_df2.reset_index()
    
    X = pd.concat([X,temp_df2.reset_index()],axis = 0)
    country_dict[i]=j
    
print(country_dict.items())
    
X = X.reset_index(drop=True)
X.tail()


dict_items([(0, 'Австралия'), (1, 'Австрия'), (2, 'Азербайджан'), (3, 'Албания'), (4, 'Алжир'), (5, 'Ангола'), (6, 'Андорра'), (7, 'Антигуа и Барбуда'), (8, 'Аргентина'), (9, 'Армения'), (10, 'Афганистан'), (11, 'Багамские острова'), (12, 'Бангладеш'), (13, 'Барбадос'), (14, 'Бахрейн'), (15, 'Беларусь'), (16, 'Белиз'), (17, 'Бельгия'), (18, 'Бенин'), (19, 'Болгария'), (20, 'Боливия'), (21, 'Босния и Герцеговина'), (22, 'Ботсвана'), (23, 'Бразилия'), (24, 'Бруней'), (25, 'Буркина-Фасо'), (26, 'Бурунди'), (27, 'Бутан'), (28, 'Великобритания'), (29, 'Венгрия'), (30, 'Венесуэла'), (31, 'Весь мир'), (32, 'Восточный Тимор'), (33, 'Вьетнам'), (34, 'Габон'), (35, 'Гаити'), (36, 'Гайана'), (37, 'Гамбия'), (38, 'Гана'), (39, 'Гватемала'), (40, 'Гвинея'), (41, 'Гвинея-Бисау'), (42, 'Германия'), (43, 'Гондурас'), (44, 'Гренада'), (45, 'Греция'), (46, 'Грузия'), (47, 'ДР Конго'), (48, 'Дания'), (49, 'Джибути'), (50, 'Доминика'), (51, 'Доминиканская Республика'), (52, 'Египет'), (53, 'Замбия'), (54,

,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths,Rt,Rt2,Rate_conf,Rate_ill,Country,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
37783,2020-08-05,42686,28020,1028,1339,833,5,0.858,1.145,0,0,Япония,13638,1329.142857,1110.714286,11963.571429,9825.428571,91,501
37784,2020-08-06,44167,28858,1034,1481,838,6,0.881,1.135,0,0,Япония,14275,1289.000000,1146.642857,12558.142857,10399.071429,142,637
37785,2020-08-07,45764,30241,1042,1597,1383,8,1.143,1.135,0,0,Япония,14481,1361.428571,1205.785714,13133.714286,10953.714286,116,206
37786,2020-08-08,47342,31248,1042,1578,1007,0,1.173,1.136,0,0,Япония,15052,1362.571429,1261.285714,13649.000000,11512.857143,-19,571
37787,2020-08-09,48782,32867,1047,1440,1619,5,1.249,1.140,0,0,Япония,14868,1380.857143,1302.428571,14008.000000,12007.428571,-138,-184


In [6]:
# Добавляем колонки расчитанные сколько доля их в процентах от суммарного показателя в мире
mask = X.Country == 'Весь мир'
temp_df = X[mask][['Date','Confirmed','Remaining_ill']]
temp_df.index = temp_df.Date
temp_df.drop(columns =['Date'], inplace = True)
print(temp_df.tail(2))
# print(temp_df.index.get_level_values(0))
# temp_df.loc[datetime.date(2020,6,15),'Россия'].Confirmed
# for i in temp_df.index.get_level_values(0).unique()[-2:]:
#     print(temp_df.loc[i,'Россия'].Confirmed)
world_date = list(temp_df.index)
world_conf = list(temp_df.Confirmed)
world_ill = list(temp_df.Remaining_ill)
world_dict_conf = dict(zip(world_date,world_conf))
world_dict_ill = dict(zip(world_date,world_ill))

print(len(X))
print(X.shape)
print(X.loc[0,'Date'])
for i in range(len(X)):
    X.loc[i,'Rate_conf']=100*X.loc[i,'Confirmed']/world_dict_conf.get(X.loc[i,'Date'])
    X.loc[i,'Rate_ill']=100*X.loc[i,'Remaining_ill']/world_dict_ill.get(X.loc[i,'Date'])


Confirmed  Remaining_ill
Date                                
2020-08-08   19637506        6971616
2020-08-09   19861683        7014532
37788
(37788, 19)
2020-01-22


In [7]:
# Проверка расчетов процентных показателей
mask = (X.Date>=datetime.date(2020,6,13))&(X.Country.isin(['Весь мир','Россия']))

temp=X[mask]
temp[['Date','Confirmed','Country','Rate_conf','Rate_ill']].tail(20)

,Date,Confirmed,Country,Rate_conf,Rate_ill
25909,2020-07-21,782040,Россия,5.231740,3.548844
25910,2020-07-22,787890,Россия,5.173797,3.424477
25911,2020-07-23,793720,Россия,5.117068,3.324245
25912,2020-07-24,799499,Россия,5.062559,3.253420
25913,2020-07-25,805332,Россия,5.018291,3.194038
25914,2020-07-26,811073,Россия,4.987776,3.199638
25915,2020-07-27,816680,Россия,4.953278,3.212297
25916,2020-07-28,822060,Россия,4.910751,3.118423
25917,2020-07-29,827509,Россия,4.859366,3.035268
25918,2020-07-30,832993,Россия,4.812261,2.950142


In [8]:
covid_rates = pd.DataFrame()
mask = (X.Date==myday)&(X.Country!='Весь мир')
today_cases = X[mask]
#covid_rates.reset_index(drop=True)
covid_rates['По количеству текущих больных'] = today_cases.sort_values(by=['Remaining_ill'],ascending = False).Country.reset_index(drop=True)
covid_rates['По количеству новых'] = today_cases.sort_values(by=['Day_confirmed'],ascending = False).Country.reset_index(drop=True)
covid_rates['По умершим за день'] = today_cases.sort_values(by=['Day_deaths'],ascending = False).Country.reset_index(drop=True)
covid_rates['По выздоровевшим за день'] = today_cases.sort_values(by=['Day_recovered'],ascending = False).Country.reset_index(drop=True)
covid_rates.head(10)
fig =  ff.create_table(covid_rates.head(15))
fig.update_layout(
    
                  title_text ='Таблица рейтингов заболеваемости COVID-19 по странам',
                  margin = {'t':100, 'b':100},
                  title_x = 0.5,
                  title_y= 0.95,
                  title_xanchor = "center",
                  title_yanchor = "middle", 
                  
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ''
)


fig.show()


In [9]:
country_number = 5
country_towatch = list(covid_rates['По количеству текущих больных'].head(country_number))
#country_towatch.append('Турция')
#сountry_towatch.append('Танзания')
country_towatch

['США', 'Индия', 'Бразилия', 'Великобритания', 'Россия']

In [10]:
mask = (X.Date>datetime.date(2020,3,1))&(X.Country.isin(country_towatch))
#mask = (X.Region.isin(regions_towatch))

fig = px.line(X[mask], y="Day_confirmed", x="Date",
                             color="Country", line_group="Country",
              line_shape="spline", render_mode="notebook",labels={'Country':'Страна:'},
              color_discrete_map={
                "США": "red",
                "Бразилия": "magenta",
                "Россия": "goldenrod",
                "Индия": "darkblue",
                "Великобритания": "green",
                 'Испания': 'aquamarine'},
             title="График количества новых заболевших COVID-19 по 5 странам<br> с наибольшим количеством текущих больных")

fig.update_layout(
    annotations=[
            dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text="Данные: Яндекс",
            xref="paper",
            yref="paper")],
           
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ''
)


fig.show()

In [11]:
#mask = (X.Date>datetime.date(2020,5,1))&(X.Region.isin(regions_towatch))

colors_dict = { "США": "red",
                "Бразилия": "magenta",
                "Россия": "goldenrod",
                "Индия": "darkblue",
                "Великобритания": "green",
                 'Испания': 'aquamarine'}
fig = go.Figure()
for i in country_towatch:
    mask = X['Country'] == i
        
    fig.add_trace(
        
        go.Bar(
        name = f'{i}',
        x=X[mask].Date,
        y=X[mask].Rt,
        text=round(X[mask].Rt,3),
        textposition='inside',
        marker_color= colors_dict.get(i),
        
        showlegend=True))




                  
fig.update_layout(
    annotations=[
        
            dict(x=0.5,y=-0.15,
            showarrow=False,
            text="Данные : Яндекс <br> формула расчета из методических рекомендациях Роспотребнадзора.",
            xref="paper",
            yref="paper")],
                  title = "Коэффициент распространения (Rt) по 6 странам ,<br> с максимальным количеством новых заболеваний и Украине ",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.9,
                  width = 990, height = 600,template = 'gridon',
    
    
                 # xaxis_title='Коэффициент рассчитан, исходя из данных оперативного штаба и <br>' + 
                               #' формулы расчета в методических рекомендациях Роспотребнадзора. ',
    yaxis_title = ' '
)


fig.show()


In [12]:
print('Все страны по которым есть статистика:')
print('*'*40)
print(country_dict.items())
print('*'*40)
print('Рассматриваем страны: ')
for i in country_dict.keys():
    if country_dict.get(i) in country_towatch:
        print(i,country_dict.get(i))

# Украина № 162
# Весь мир № 31
# Турция № 159
#  Танзания № 155
my_country = country_dict[162]
print('Выбрали страну: ',my_country)


Все страны по которым есть статистика:
****************************************
dict_items([(0, 'Австралия'), (1, 'Австрия'), (2, 'Азербайджан'), (3, 'Албания'), (4, 'Алжир'), (5, 'Ангола'), (6, 'Андорра'), (7, 'Антигуа и Барбуда'), (8, 'Аргентина'), (9, 'Армения'), (10, 'Афганистан'), (11, 'Багамские острова'), (12, 'Бангладеш'), (13, 'Барбадос'), (14, 'Бахрейн'), (15, 'Беларусь'), (16, 'Белиз'), (17, 'Бельгия'), (18, 'Бенин'), (19, 'Болгария'), (20, 'Боливия'), (21, 'Босния и Герцеговина'), (22, 'Ботсвана'), (23, 'Бразилия'), (24, 'Бруней'), (25, 'Буркина-Фасо'), (26, 'Бурунди'), (27, 'Бутан'), (28, 'Великобритания'), (29, 'Венгрия'), (30, 'Венесуэла'), (31, 'Весь мир'), (32, 'Восточный Тимор'), (33, 'Вьетнам'), (34, 'Габон'), (35, 'Гаити'), (36, 'Гайана'), (37, 'Гамбия'), (38, 'Гана'), (39, 'Гватемала'), (40, 'Гвинея'), (41, 'Гвинея-Бисау'), (42, 'Германия'), (43, 'Гондурас'), (44, 'Гренада'), (45, 'Греция'), (46, 'Грузия'), (47, 'ДР Конго'), (48, 'Дания'), (49, 'Джибути'), (50, 'До

In [13]:
mask = X.Country ==my_country
#mask = (X.Date>datetime.date(2020,5,1)&(X.Region.isin(regions_towatch))
#mask = (X.Date>datetime.date(2020,5,1)&(X.Region==my_region)

fig = px.bar(X[mask], x='Date', 
             y="Day_confirmed",color ='Day_confirmed',
             title=f'{my_country}: '+'график количества новых заболевших COVID-19 по датам',
              color_continuous_scale= px.colors.sequential.BuPu,text = 'Day_confirmed',
             labels = {'Day_confirmed' : "Новые заболевшие",'Remaining_ill':'Текущие больные'})
fig.add_trace(
    go.Scatter(
        name='14-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA14_dayconfirmed'],
        mode="lines",
        
        line=go.scatter.Line(color="green"),
        showlegend=True)
)
fig.add_trace(
    go.Scatter(
        name = '7-дневная скользящая средняя',
        x=X[mask]['Date'],
        y=X[mask]['MA7_dayconfirmed'],
        mode="lines",
        
        line=go.scatter.Line(color="red"),
        showlegend=True)
)
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=1355,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=1572,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-90),
        dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text="Данные: Яндекс",
            xref="paper",
            yref="paper"),
        
                ],
    
    
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.10,legend_y = 0.95,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ' '
)


fig.show()


In [14]:
mask = X.Country ==my_country
#mask = (X.Date>datetime.date(2020,5,1))&(X.Region==my_region)
Y = X[mask]

Y['base'] = Y['Change_dayconf']*Y['Change_dayconf'].apply(lambda d:0 if d>0 else 1)
Y['Date']= pd.to_datetime(Y.Date)
fig = go.Figure()
fig.add_trace(
              go.Bar(x = Y.Date,y=Y.Change_dayconf,text = Y['Change_dayconf'],textposition = 'inside',
                
                #marker_color=Y['Change_dayconf'].apply(lambda d:'crimson' if d>0 else 'green'),
                 marker_color = Y['Change_dayconf'],marker_colorscale = 'Temps',marker_colorbar ={'tickmode':'auto',
                                    'title':{'text':'Изменение','side':'top'}}
                    )   
                )
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=10594,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=-429,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=60),
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title=f'{my_country}: динамика новых заболевших COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Данные: Яндекс',
    yaxis_title = ' '
)


fig.show()


In [15]:
mask = X.Country ==my_country
#mask = (X.Date>'2020-05-01')&(X.Region.isin(regions_towatch))

fig = px.bar(X[mask], x='Date', y="Remaining_ill",color ='Remaining_ill',
             title=f'{my_country}: '+'график количества текущих больных COVID-19 по датам',
              color_continuous_scale= px.colors.sequential.Purp,text = 'Remaining_ill',
            labels = {'Day_confirmed' : "Новые заболевшие",'Remaining_ill':'Текущие больные'})
fig.add_trace(
    go.Scatter(
        name='14-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA14_remaining_ill'],
        mode="lines",
        
        line=go.scatter.Line(color="green"),
        showlegend=True)
)

fig.add_trace(
    go.Scatter(
        name = '7-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA7_remaining_ill'],
        mode="lines",
        
        line=go.scatter.Line(color="red"),
        showlegend=True)
)
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=10594,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=85824,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.15,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Текущие больные = количество выявленных - количество выздоровевших - количество умерших'+
                                '<br>Данные: Яндекс',
    yaxis_title = ' '
)


fig.show()


In [16]:
mask = X.Country ==my_country
#mask = (X.Date>'2020-05-01')&(X.Region==my_region)
Y = X[mask]

#Y['base'] = Y['Change_dayconf']*Y['Change_dayconf'].apply(lambda d:0 if d>0 else 1)
Y['Date']= pd.to_datetime(Y.Date)
fig = go.Figure()
fig.add_trace(
              go.Bar(x = Y.Date,y=Y.Change_remill,text = Y['Change_remill'],textposition = 'inside',
                
                #marker_color=Y['Change_remill'].apply(lambda d:'crimson' if d>0 else 'green'),
                marker_color = Y['Change_remill'],marker_colorscale = 'Temps',marker_colorbar ={'tickmode':'auto',
                                    'title':{'text':'Изменение','side':'top'}})     
                )
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=10594,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=-3877,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=-20,
#             ay=40),
#         dict(
#             x='2020-05-20',
#             y=-2816,
#             xref="x",
#             yref="y",
#             text="20 мая",
#             showarrow=True,
#             arrowhead=2,
#             ax=-20,
#             ay=40),
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title=f'{my_country}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Текущие больные = количество выявленных - количество выздоровевших - количество умерших'+
                               '<br>Данные : Яндекс',
    yaxis_title = ' '
)


fig.show()


In [17]:
mask = (X.Date>=datetime.date(2020,7,26))&(X.Country.isin(country_towatch))


# #mask = (X.Date>='2020-06-04')

Z = X[mask].groupby(['Country','Date']).Rt.sum().unstack().reset_index()
Z.dropna(inplace=True)
d = Z.drop('Country',axis = 1).values

# my_map= {'Москва':'2.Москва','Московская обл.':'3.Московская обл.','Нижегородская обл.':'5.Нижегородская обл.',
#                     'Россия':'1.Россия', 'Санкт-Петербург':'4.Санкт-Петербург'}
# Z.Region = Z.Region.map(my_map)

print(d)
Z.head()

[[1.255 0.819 0.622 0.672 0.997 1.421 1.607 1.15  0.734 0.633 0.673 0.984
  1.513 1.51  1.166]
 [1.322 1.255 1.08  0.969 0.942 1.036 1.196 1.162 1.156 1.016 0.991 1.074
  1.031 1.128 1.144]
 [1.222 1.13  1.071 1.02  1.012 1.125 1.125 1.129 1.11  0.983 0.976 1.007
  1.054 1.153 1.172]
 [0.979 0.981 0.969 0.952 0.946 0.949 0.968 0.982 0.988 0.978 0.965 0.961
  0.958 0.977 0.986]
 [1.02  0.947 0.877 0.887 0.99  1.081 1.085 0.972 0.837 0.769 0.769 0.893
  1.044 1.086 1.087]]


Date,Country,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09
0,Бразилия,1.255,0.819,0.622,0.672,0.997,1.421,1.607,1.150,0.734,0.633,0.673,0.984,1.513,1.510,1.166
1,Великобритания,1.322,1.255,1.080,0.969,0.942,1.036,1.196,1.162,1.156,1.016,0.991,1.074,1.031,1.128,1.144
2,Индия,1.222,1.130,1.071,1.020,1.012,1.125,1.125,1.129,1.110,0.983,0.976,1.007,1.054,1.153,1.172
3,Россия,0.979,0.981,0.969,0.952,0.946,0.949,0.968,0.982,0.988,0.978,0.965,0.961,0.958,0.977,0.986
4,США,1.020,0.947,0.877,0.887,0.990,1.081,1.085,0.972,0.837,0.769,0.769,0.893,1.044,1.086,1.087


In [18]:
x1=list(Z.columns[1:])
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
y=list(Z.Country)
print(list(x1))
print(x2)

[datetime.date(2020, 7, 26), datetime.date(2020, 7, 27), datetime.date(2020, 7, 28), datetime.date(2020, 7, 29), datetime.date(2020, 7, 30), datetime.date(2020, 7, 31), datetime.date(2020, 8, 1), datetime.date(2020, 8, 2), datetime.date(2020, 8, 3), datetime.date(2020, 8, 4), datetime.date(2020, 8, 5), datetime.date(2020, 8, 6), datetime.date(2020, 8, 7), datetime.date(2020, 8, 8), datetime.date(2020, 8, 9)]
['26.7.2020', '27.7.2020', '28.7.2020', '29.7.2020', '30.7.2020', '31.7.2020', '1.8.2020', '2.8.2020', '3.8.2020', '4.8.2020', '5.8.2020', '6.8.2020', '7.8.2020', '8.8.2020', '9.8.2020']


In [19]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,    
    
                  xaxis_title='Динамика коэффициента распространения Rt по странам',
               #   yaxis_title = 'Регион '
)

fig.show()



In [20]:
mask = (X.Date>=datetime.date(2020,7,26))&(X.Country.isin(country_towatch))


# #mask = (X.Date>='2020-06-04')

Z = X[mask].groupby(['Country','Date']).Rt2.sum().unstack().reset_index()
Z.dropna(inplace=True)
d = Z.drop('Country',axis = 1).values

# my_map= {'Москва':'2.Москва','Московская обл.':'3.Московская обл.','Нижегородская обл.':'5.Нижегородская обл.',
#                     'Россия':'1.Россия', 'Санкт-Петербург':'4.Санкт-Петербург'}
# Z.Region = Z.Region.map(my_map)

print(d)
Z.head()

[[1.098 1.092 1.08  1.07  1.068 1.074 1.081 1.083 1.076 1.067 1.058 1.057
  1.063 1.068 1.07 ]
 [1.009 1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.011 1.011 1.011 1.011
  1.011 1.011 1.012]
 [1.157 1.157 1.153 1.149 1.144 1.143 1.142 1.142 1.141 1.136 1.13  1.126
  1.124 1.125 1.126]
 [1.03  1.03  1.029 1.029 1.028 1.027 1.027 1.027 1.026 1.026 1.026 1.025
  1.025 1.025 1.024]
 [1.07  1.068 1.065 1.062 1.061 1.061 1.061 1.06  1.057 1.052 1.048 1.046
  1.046 1.047 1.047]]


Date,Country,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09
0,Бразилия,1.098,1.092,1.080,1.070,1.068,1.074,1.081,1.083,1.076,1.067,1.058,1.057,1.063,1.068,1.070
1,Великобритания,1.009,1.010,1.010,1.010,1.010,1.010,1.010,1.010,1.011,1.011,1.011,1.011,1.011,1.011,1.012
2,Индия,1.157,1.157,1.153,1.149,1.144,1.143,1.142,1.142,1.141,1.136,1.130,1.126,1.124,1.125,1.126
3,Россия,1.030,1.030,1.029,1.029,1.028,1.027,1.027,1.027,1.026,1.026,1.026,1.025,1.025,1.025,1.024
4,США,1.070,1.068,1.065,1.062,1.061,1.061,1.061,1.060,1.057,1.052,1.048,1.046,1.046,1.047,1.047


In [21]:
x1=list(Z.columns[1:])
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
y=list(Z.Country)
print(list(x1))
print(x2)

[datetime.date(2020, 7, 26), datetime.date(2020, 7, 27), datetime.date(2020, 7, 28), datetime.date(2020, 7, 29), datetime.date(2020, 7, 30), datetime.date(2020, 7, 31), datetime.date(2020, 8, 1), datetime.date(2020, 8, 2), datetime.date(2020, 8, 3), datetime.date(2020, 8, 4), datetime.date(2020, 8, 5), datetime.date(2020, 8, 6), datetime.date(2020, 8, 7), datetime.date(2020, 8, 8), datetime.date(2020, 8, 9)]
['26.7.2020', '27.7.2020', '28.7.2020', '29.7.2020', '30.7.2020', '31.7.2020', '1.8.2020', '2.8.2020', '3.8.2020', '4.8.2020', '5.8.2020', '6.8.2020', '7.8.2020', '8.8.2020', '9.8.2020']


In [22]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,    
    
                  xaxis_title='Динамика коэффициента распространения Rt2 по странам',
               #   yaxis_title = 'Регион '
)

fig.show()


In [23]:
mask = (X.Date>=datetime.date(2020,7,26))&(X.Country.isin(country_towatch))

Z = X[mask].groupby(['Country','Date'])['Day_confirmed'].sum().unstack().reset_index()
Z.dropna(inplace=True)
d = Z.drop('Country',axis = 1).values
Z.head()

Date,Country,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09
0,Бразилия,24578,23284,40816,69074,57837,52383,45392,25800,16641,51603,57152,53139,50230,49970,23010
1,Великобритания,750,688,553,802,847,883,769,747,942,675,906,964,900,765,1113
2,Индия,49981,44457,51596,50294,52783,61242,54735,52972,52050,52509,56282,62538,61537,64399,62064
3,Россия,5741,5607,5380,5449,5484,5468,5429,5387,5364,5121,5186,5239,5191,5185,5155
4,США,54953,56414,65869,70776,68033,67092,58485,47580,45368,57540,52810,59692,58173,56174,46935


In [24]:



x1=list(Z.columns[1:])
y=list(Z.Country)
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
print(x2)


['26.7.2020', '27.7.2020', '28.7.2020', '29.7.2020', '30.7.2020', '31.7.2020', '1.8.2020', '2.8.2020', '3.8.2020', '4.8.2020', '5.8.2020', '6.8.2020', '7.8.2020', '8.8.2020', '9.8.2020']


In [25]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,
    
    
                  xaxis_title='Динамика новых заболевших по странам',
               #   yaxis_title = 'Регион '
)

fig.show()

In [26]:
mask = (X.Date==myday)&(X.Country != 'Весь мир')
#mask = (X.Date==myday)
#Делаем выборку по доле регина в общих случаях выявления заболеваний по России
df=X[mask].sort_values(by=['Rate_conf'],ascending = False)
#Далее заменяем все имена если регион набрал менее 1% в общих случаях по России
df2 = X[mask].reset_index(drop=True)
print(len(df2))

for i in range(len(df2)):
    if df2.loc[i,'Rate_conf']<1:
        df2.loc[i,'Country']='Страны менее 1%'
df.head(15)

187


,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths,Rt,Rt2,Rate_conf,Rate_ill,Country,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
26531,2020-08-09,5044864,1656864,162938,46935,13746,515,1.087,1.047,25.399982,45.976866,США,3225062,53813.142857,57924.357143,3.130662e+06,3.029462e+06,-9239,32674
4823,2020-08-09,3035422,2356983,101049,23010,35446,572,1.166,1.070,15.282804,8.231340,Бразилия,577390,43106.428571,44023.642857,5.740410e+05,5.619894e+05,-26960,-13008
11858,2020-08-09,2215074,1535743,44386,62064,54859,1007,1.172,1.126,11.152499,9.051851,Индия,634945,58768.428571,55675.571429,6.083153e+05,5.747354e+05,-2335,6198
25928,2020-08-09,885718,692059,14903,5155,3203,76,0.986,1.024,4.459431,2.548367,Россия,178756,5205.857143,5331.785714,1.815110e+05,1.866134e+05,-30,1876
36983,2020-08-09,559859,411474,10408,6671,6906,198,1.128,1.056,2.818789,1.967016,ЮАР,137977,6910.571429,8173.285714,1.429630e+05,1.524614e+05,-1041,-433
20903,2020-08-09,480278,384432,52298,4376,7307,292,1.104,1.056,2.418113,0.620825,Мексика,43548,5890.285714,6411.571429,4.511771e+04,4.737500e+04,-2119,-3223
25124,2020-08-09,478024,324020,21072,14149,4849,423,0.758,1.064,2.406765,1.895094,Перу,132932,7024.857143,7290.214286,1.234456e+05,1.131097e+05,14149,8877
15878,2020-08-09,387481,212688,12842,10611,8097,302,1.057,1.123,1.950897,2.308793,Колумбия,161951,9975.714286,9893.214286,1.522643e+05,1.401353e+05,945,2212
34973,2020-08-09,373056,345826,10077,2033,1693,66,1.179,1.021,1.878270,0.244535,Чили,17153,1903.571429,1947.571429,1.695457e+04,1.744114e+04,-165,274
12662,2020-08-09,326712,284371,18427,2020,2249,163,0.860,1.032,1.644936,0.340921,Иран,23914,2467.857143,2538.571429,2.448486e+04,2.401900e+04,-105,-392


In [27]:



fig = px.pie(df2, values='Confirmed', names='Country',color_discrete_sequence=px.colors.sequential.Rainbow,
            title=f'Всего выявленных случаев заболевания COVID-19 в мире: {df2.Confirmed.sum()}<br>'+
            f'(на дату: {myday})'
            ,
            width = 990, height=1000)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
        
                #  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  margin = {'t':150, 'b':0},
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.98,
                     
)

fig.show()



In [28]:
mask = (X.Date==myday)&(X.Country !='Весь мир')
#mask = (X.Date==myday)
#Делаем выборку по доле регина в общих случаях выявления заболеваний по России
df=X[mask].sort_values(by=['Rate_ill'],ascending = False)
#Далее заменяем все имена если регион набрал менее 1% в общих случаях по России
df2 = X[mask].reset_index(drop=True)
print(len(df2))

for i in range(len(df2)):
    if df2.loc[i,'Rate_ill']<1:
        df2.loc[i,'Country']='Страны менее 1%'
df.head(20)

187


,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths,Rt,Rt2,Rate_conf,Rate_ill,Country,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
26531,2020-08-09,5044864,1656864,162938,46935,13746,515,1.087,1.047,25.399982,45.976866,США,3225062,53813.142857,57924.357143,3.130662e+06,3.029462e+06,-9239,32674
11858,2020-08-09,2215074,1535743,44386,62064,54859,1007,1.172,1.126,11.152499,9.051851,Индия,634945,58768.428571,55675.571429,6.083153e+05,5.747354e+05,-2335,6198
4823,2020-08-09,3035422,2356983,101049,23010,35446,572,1.166,1.070,15.282804,8.231340,Бразилия,577390,43106.428571,44023.642857,5.740410e+05,5.619894e+05,-26960,-13008
5828,2020-08-09,312574,1451,46659,1113,2,8,1.144,1.012,1.573754,3.770230,Великобритания,264464,895.000000,825.285714,2.618491e+05,2.591264e+05,348,1103
25928,2020-08-09,885718,692059,14903,5155,3203,76,0.986,1.024,4.459431,2.548367,Россия,178756,5205.857143,5331.785714,1.815110e+05,1.866134e+05,-30,1876
15878,2020-08-09,387481,212688,12842,10611,8097,302,1.057,1.123,1.950897,2.308793,Колумбия,161951,9975.714286,9893.214286,1.522643e+05,1.401353e+05,945,2212
36983,2020-08-09,559859,411474,10408,6671,6906,198,1.128,1.056,2.818789,1.967016,ЮАР,137977,6910.571429,8173.285714,1.429630e+05,1.524614e+05,-1041,-433
13265,2020-08-09,314362,150376,28503,0,0,0,0.498,1.049,1.582756,1.931462,Испания,135483,3691.428571,2995.785714,1.294951e+05,1.176981e+05,0,0
1808,2020-08-09,246499,108242,4606,4688,0,83,1.070,1.130,1.241078,1.905345,Аргентина,133651,6368.571429,5998.071429,1.230630e+05,1.119412e+05,-1446,4605
25124,2020-08-09,478024,324020,21072,14149,4849,423,0.758,1.064,2.406765,1.895094,Перу,132932,7024.857143,7290.214286,1.234456e+05,1.131097e+05,14149,8877


In [29]:


fig = px.pie(df2, values='Remaining_ill', names='Country',color_discrete_sequence=px.colors.sequential.Rainbow,
            title=f'Всего текущих больных COVID-19 в мире: {df2.Remaining_ill.sum()}<br>'+
            f'(на дату: {myday})'
             ,
            width = 990, height=1100)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
        
                #  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  margin = {'t':120, 'b':0},
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.98,
                     
)

fig.show()


In [30]:
covid_rates = pd.DataFrame()
mask = (X.Date>=datetime.date(2020,8,2))&(X.Date<=datetime.date(2020,8,9))&(X.Country.isin(country_towatch))&(X.Country!='Весь мир')
                                      
box_cases1 = X[mask].groupby(['Country','Date'])['Day_confirmed'].sum()
box_cases2 = X[mask].groupby(['Country','Date'])['Day_recovered'].sum()
for i in country_dict.keys():
    if country_dict.get(i) in country_towatch:
        print(i,country_dict.get(i))

23 Бразилия
28 Великобритания
58 Индия
128 Россия
131 США


In [31]:
new_df1=box_cases1.unstack().T
new_df1.tail(20)

Country,Бразилия,Великобритания,Индия,Россия,США
Date,,,,,
2020-08-02,25800,747,52972,5387,47580
2020-08-03,16641,942,52050,5364,45368
2020-08-04,51603,675,52509,5121,57540
2020-08-05,57152,906,56282,5186,52810
2020-08-06,53139,964,62538,5239,59692
2020-08-07,50230,900,61537,5191,58173
2020-08-08,49970,765,64399,5185,56174
2020-08-09,23010,1113,62064,5155,46935


In [32]:
color_dict = {
                "США": "red",
                "Бразилия": "magenta",
                "Индия": "goldenrod",
                "Великобритания": "darkblue",
                "Россия": "green",
                 }

cols = list(new_df1.columns)
#cols = cols[:-1]

fig = go.Figure()
for i in cols:
    fig.add_trace(
    go.Box(x=new_df1[i],name = i,boxmean = True,marker_color = color_dict.get(i)))

fig.update_layout(title_text="Статистика по новым заболевшим за неделю",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "top",
                  xaxis_title='Новые заболевшие', yaxis_title='Страны',width = 990, height = 600)
fig.show()

In [33]:
new_df2=box_cases2.unstack().T
new_df2.tail(20)

Country,Бразилия,Великобритания,Индия,Россия,США
Date,,,,,
2020-08-02,24894,3,40574,3645,6804
2020-08-03,36100,1,44306,3411,44757
2020-08-04,58508,0,51706,7863,15533
2020-08-05,32877,1,46121,7534,48872
2020-08-06,40181,1,49769,7300,20773
2020-08-07,41757,0,48900,7209,25246
2020-08-08,49238,2,53879,6578,19248
2020-08-09,35446,2,54859,3203,13746


In [34]:

cols = list(new_df2.columns)
#cols = cols[:-1]

fig = go.Figure()
for i in cols:
    fig.add_trace(
    go.Box(x=new_df2[i],name = i,boxmean=True,marker_color = color_dict.get(i)))

fig.update_layout(title_text="Статистика по выздоровевшим за неделю",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "top",
                  xaxis_title='Выздоровевшие', yaxis_title='Регионы',width = 990, height = 600)
fig.show()

In [35]:
from plotly.subplots import make_subplots

# fig = tools.make_subplots(rows = 1, cols = 2, specs = [[{}, {}]],
#                           horizontal_spacing = 0.05)


fig = make_subplots(rows=2, cols=1,specs = [[{}],[{}]],vertical_spacing = 0.03,shared_xaxes=True,
                  subplot_titles=("Box-график распределения по новым заболевшим(в день) за неделю:",
                                 "Box-график распределения по выздоровевшим(в день) за неделю:"))
#subplot_titles=("Статистика по новым заболевшим за последние 2 недели","Статистика по выздоровевшим за последние 2 недели")

for i in cols:
    fig.add_trace(
    go.Box(x=new_df1[i],name = i,boxmean = True,marker_color = color_dict.get(i)),row=1,col=1)

for i in cols:
    fig.add_trace(
    go.Box(x=new_df2[i],name = i,boxmean=True,marker_color = color_dict.get(i)),row=2,col=1)


fig.update_layout(
#     annotations=[
#         dict(
#             x=0.5,
#             y=-0.15,
#             showarrow=False,
#             text="Ветикальная пунктирная линия - среднее, Вертикальная сплошная - медиана распоеделения",
#             xref="paper",
#             yref="paper"),
        
#                 ],
    
    
                  width = 990, height = 1200,
    
    
                 showlegend=False,template ='ggplot2'
)

#fig.update_layout(height=1000, width=800,showlegend=False)
fig.show()